In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.transforms import v2

In [3]:
from google.colab import drive
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/cifar-10-batches-py'
transform = transforms.Compose([
    transforms.ToTensor(),
   transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_dataset = datasets.CIFAR10(data_path, train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(data_path, train=False, download=True, transform=transform)

training_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)


x_train = torch.stack([train_dataset[i][0] for i in range(len(train_dataset))]).numpy()
y_train = np.array([train_dataset[i][1] for i in range(len(train_dataset))])

x_test = torch.stack([test_dataset[i][0] for i in range(len(test_dataset))]).numpy()
y_test = np.array([test_dataset[i][1] for i in range(len(test_dataset))])

x_train = x_train.reshape(x_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)
# Επιλογή των κλάσεων
selected_classes = [1, 9]
train_mask = np.isin([train_dataset[i][1] for i in range(len(train_dataset))], selected_classes)
test_mask = np.isin([test_dataset[i][1] for i in range(len(test_dataset))], selected_classes)

# Επιλογή των δεδομένων με βάση τις μάσκες
x_train = torch.stack([train_dataset[i][0] for i in range(len(train_dataset))])[train_mask][:1000]
y_train = np.array([train_dataset[i][1] for i in range(len(train_dataset))])[train_mask][:1000]
x_test = torch.stack([test_dataset[i][0] for i in range(len(test_dataset))])[test_mask][:200]
y_test = np.array([test_dataset[i][1] for i in range(len(test_dataset))])[test_mask][:200]

train_filter = np.isin(y_train, selected_classes)
test_filter = np.isin(y_test, selected_classes)

x_train_filtered = x_train[train_filter]
y_train_filtered = y_train[train_filter]
x_test_filtered = x_test[test_filter]
y_test_filtered = y_test[test_filter]

# Μετατροπή ετικετών σε -1 και 1
y_train_filtered = np.where(y_train_filtered == selected_classes[0], -1, 1)
y_test_filtered = np.where(y_test_filtered == selected_classes[0], -1, 1)
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

Mounted at /content/drive
Files already downloaded and verified
Files already downloaded and verified


In [4]:

class Neural:
    def __init__(self, input, hidden, output):
        self.input = input
        self.hidden = hidden
        self.output = output
        # Αρχικοποίηση τυχαίων βαρών και bias
        self.weights_input_hidden = np.random.randn(self.input, self.hidden)
        self.weights_output_hidden = np.random.randn(self.hidden, self.output)
        self.bias_hidden = np.zeros((1, self.hidden))
        self.bias_output = np.zeros((1, self.output))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    # Μέθοδος feedforward
    def feedforward(self, X):

        self.net_hidden = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = self.sigmoid(self.net_hidden)

        self.net_output = np.dot(self.hidden_output, self.weights_output_hidden) + self.bias_output
        self.predicted_output = self.sigmoid(self.net_output)
        return self.predicted_output

    # Υπολογισμός Hinge Loss
    def hinge_loss(self, y, predicted_output):
        return np.mean(np.maximum(0, 1 - y.reshape(-1, 1) * predicted_output))

    # Μέθοδος backpropagation
    def back(self, X, y, learning_rate):

        output_error = y.reshape(-1, 1) - self.predicted_output

        # Υπολογισμός του delta για το output layer
        delta_k = -y.reshape(-1, 1) * (self.predicted_output < 1)  # Hinge loss gradient

        # Υπολογισμός του delta για το hidden layer
        delta_j = self.sigmoid_derivative(self.hidden_output) * np.dot(delta_k, self.weights_output_hidden.T)

        # Ενημέρωση των w και των bias
        self.weights_output_hidden += np.dot(self.hidden_output.T, delta_k) * learning_rate
        self.bias_output += np.sum(delta_k, axis=0, keepdims=True) * learning_rate

        self.weights_input_hidden += np.dot(X.T, delta_j) * learning_rate
        self.bias_hidden += np.sum(delta_j, axis=0, keepdims=True) * learning_rate

    # Μέθοδος εκπαίδευσης
    def train(self, X, y, epochs, learning_rate):
        for epoch in range(epochs):
            output = self.feedforward(X)
            self.back(X, y, learning_rate)

            loss = self.hinge_loss(y, self.predicted_output)  # Υπολογισμός της Hinge Loss





In [5]:
nn = Neural(input=32*32*3, hidden=64, output=1)
# Preprocessing ta dedomena
# Flatten ta eikona se 3072 dis diastaseis (3 * 32 * 32)
x_train_filtered_flat = x_train_filtered.reshape(x_train_filtered.shape[0], -1)
x_test_filtered_flat = x_test_filtered.reshape(x_test_filtered.shape[0], -1)

# Ekpaidefsi tou montelou
nn.train(x_train_filtered_flat, y_train_filtered, epochs=100, learning_rate=0.001)

# Ypologismos Training Accuracy
train_predictions = nn.feedforward(x_train_filtered_flat)
train_predictions = np.where(train_predictions > 0.5, 1, -1)
train_accuracy = np.mean(train_predictions == y_train_filtered)
print(f"Training Accuracy: {train_accuracy * 100:.2f}%")

Training Accuracy: 48.44%
